# Реализация наивного Байесовского классификатора

In [16]:
import pandas as pd

filename = 'data/sms_spam_collection.tar.gz'

df = pd.read_csv(
    filename,
    compression='gzip',
    header=1,
    sep='\t',
    encoding='utf8',
    names=['class', 'sms_text'],
    error_bad_lines=False
)


df.head(3)

,class,sms_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,U dun say so early hor... U c already then say...
2,ham,"Nah I don't think he goes to usf, he lives aro..."


Проверяем, сколько у нас всего объектов в датасете

In [2]:
num_objects, num_features = df.shape
print(num_objects, num_features)

5571 2


Целевая переменная (target) в столбце `class`

In [15]:
df['class'].head()

0    spam
1     ham
2     ham
3    spam
4     ham
Name: class, dtype: object

Демонстрация того, как получить булеву маску для датафрейма

In [4]:
SPAM_CLASS = 'spam'
NOT_SPAM_CLASS = 'ham'

df['class'] == SPAM_CLASS

0        True
1       False
2       False
3        True
4       False
        ...  
5566    False
5567    False
5568    False
5569    False
5570    False
Name: class, Length: 5571, dtype: bool

Использование булевой маски для фильтрации датафрейма 

In [20]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()
notspam_sms_num = (df['class'] == NOT_SPAM_CLASS).sum()

print(f'spam sms: {spam_sms_num}, not spam sms {notspam_sms_num}')

spam sms: 747, not spam sms 4823


In [21]:
df['class'].value_counts()['ham']

4823

##### Задача

считаем вероятности классов

In [24]:
sms_num = spam_sms_num + notspam_sms_num

# априорная вероятность класса спам
p_spam = spam_sms_num / sms_num

# априорная вероятность класса не спам
p_notspam =  notspam_sms_num / sms_num

print(f'{p_spam:.4f}, {p_notspam:.4f}')

0.1341, 0.8659


Пример обработки текстовой информации - приводим к нижнему регистру

In [7]:
test_word = 'Free'.lower()

test_word

'free'

In [8]:
sms_example = df['sms_text'].values[0]

sms_example

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

Пишем полезные сниппеты для трансформации текста

In [32]:
# удаляем знаки препинания
import string

' '.join([
    str(char)
    for char in sms_example
    if char not in string.punctuation
])

print(string.punctuation)

sms_example = ' '.join([
    char
    for char in sms_example
    if char not in string.punctuation
])

sms_example

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


'F   r   e   e       e   n   t   r   y       i   n       2       a       w   k   l   y       c   o   m   p       t   o       w   i   n       F   A       C   u   p       f   i   n   a   l       t   k   t   s       2   1   s   t       M   a   y       2   0   0   5       T   e   x   t       F   A       t   o       8   7   1   2   1       t   o       r   e   c   e   i   v   e       e   n   t   r   y       q   u   e   s   t   i   o   n   s   t   d       t   x   t       r   a   t   e   T   C   s       a   p   p   l   y       0   8   4   5   2   8   1   0   0   7   5   o   v   e   r   1   8   s'

In [33]:
# приводим слова к нижнему регистру

#.lower()
sms_example = ' '.join([
    word.lower()
    for word in sms_example.split(sep=' ')
])

sms_example

'f   r   e   e       e   n   t   r   y       i   n       2       a       w   k   l   y       c   o   m   p       t   o       w   i   n       f   a       c   u   p       f   i   n   a   l       t   k   t   s       2   1   s   t       m   a   y       2   0   0   5       t   e   x   t       f   a       t   o       8   7   1   2   1       t   o       r   e   c   e   i   v   e       e   n   t   r   y       q   u   e   s   t   i   o   n   s   t   d       t   x   t       r   a   t   e   t   c   s       a   p   p   l   y       0   8   4   5   2   8   1   0   0   7   5   o   v   e   r   1   8   s'

Объединяем сниппеты в функцию

In [29]:
import string

def text_preprocess(sms_text: str) -> str:
    """Преобразование текста для анализа"""
    text_no_punctuation = ''.join([
        char
        for char in sms_text
        if char not in string.punctuation
    ])
    text_lowercase = ' '.join([
        word.lower()
        for word in text_no_punctuation.split(sep=' ')
    ])
    
    return text_lowercase


sms_example = df['sms_text'].values[0]

print(text_preprocess(sms_example))

free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s


Трансформируем каждую строчку датафрейма

In [34]:
df = df.assign(
    processed_text=df['sms_text'].astype(str).apply(text_preprocess)
)

df.head()

,class,sms_text,processed_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
1,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
2,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...
3,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darling its been 3 weeks now...
4,ham,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me t...


#### Задача

Находим вероятность встретить слово в каждом из классов - это наша основная "фича" в наивном байесовском классификаторе

In [36]:
# вероятность встретить слово в спам смс
# посчитайте, сколько раз test_word встречаемся в документах класса spam
spam_test_word_entries = df[df['class'] == 'spam']['processed_text'].str.contains(test_word).sum()

# вероятность встретить слово в не-спам смс
# посчитайте, сколько раз test_word встречаемся в документах класса spam
notspam_test_word_entries = df[df['class'] == 'ham']['processed_text'].str.contains(test_word).sum()
    
print(f'P(word="{test_word}"|class=spam)={spam_test_word_entries/spam_sms_num:.4f}')
print(f'P(word="{test_word}"|class=not_spam)={notspam_test_word_entries/notspam_sms_num:.4f}')

P(word="free"|class=spam)=0.2664
P(word="free"|class=not_spam)=0.0137


### Вывод

слово "free" встречается в спам смс с вероятностью $26.6\%$, а в не-спаме с вероятностью $1.3\%$ - т.е. это слово является хорошим "маркером" спама

# Реализовать классификатор

Аналогично тому, как посчитали вероятности встретить слово `free` в каждом классе (спам/не спам) 
* в функции `fit()` подсчитать такие вероятности для каждого слова
* в функции `predict()` по формуле байеса (см. лекцию) вычислять вероятность принадлежности входного текста к каждому из классов

Результат предсказания - класс, вероятность принадлежности к которому больше

In [41]:
from typing import List
from copy import deepcopy
from collections import defaultdict

import numpy as np
from tqdm import tqdm

"""имплементация наивного байесовского классификатора"""
class NaiveBayes:
    def __init__(self):
        
        self.labels = [NOT_SPAM_CLASS, SPAM_CLASS]
        self.class_labels_proba = None  # априорная вероятность класса, словарь
        self.prior_word_proba = None  # частоты фичей (токенов)
    
    def _set_labels_prior_proba(self, data: list, target: list) -> None:
        """Вычисление априорной вероятности классов
        
        Вызов функции должен инициализировать массив self.class_labels_proba
        
        """
#         class_labels_proba = dict.fromkeys(self.labels, 0.0)

        # ----------------- #
        # -- Ваш код тут -- #
        class_labels_proba = dict(pd.Series(target).value_counts(normalize=True))
        # ----------------- #
        
        self.class_labels_proba = class_labels_proba
    
    def _tokenize_text(self, text) -> list:
        """Функция, которая разобьёт входной текст на токены
        
        Токены вернуть в виде списка"""
        
#         tokens = []
        # ----------------- #
        # -- Ваш код тут -- #
        text_no_punctuation = ''.join([
        char
        for char in str(text)
        if char not in string.punctuation
        ])
    
        tokens = [
        word.lower()
        for word in text_no_punctuation.split(' ')
        ]
        try:
            tokens.remove('')
        except:
            pass
        # ----------------- #
        
        return tokens
    
    def _set_word_prior_proba(self, data, target):
        """Вычисляем априорную вероятность токенов в классе
        
        Заполняем словарь self.prior_word_proba[label][token]
        
        """
        word_proba_dict_by_class = dict.fromkeys(self.labels)

        # ----------------- #
        for label in self.labels:
            word_proba_dict_by_class[label] = dict()
            
        df = pd.DataFrame({'data':data,
                           'target':target})
        
        unique_tokens = set()
        for text in data:
            for token in self._tokenize_text(text):
                
                unique_tokens.add(token)
        
        unique_tokens.remove('') # FIXME: dont know why '' is here.
        
        for token in tqdm(unique_tokens, total=len(unique_tokens)):
            for label in self.labels:
                word_proba = df[df['target'] == label]['data'].str.lower().str.contains(token)\
                            .value_counts(normalize=True).get(True, 0)
                word_proba_dict_by_class[label][token] = word_proba
                
        
        # -- Ваш код тут -- #
        # ----------------- #
        
        self.prior_word_proba = word_proba_dict_by_class

    def fit(self, data: list, target: list):
        """Обучение статистик по датасету

        :param data: массив документов, каждый документ - объект типа str
        :param target: массив меток объектов
        :return:
        """
        if not isinstance(data, list):
            raise ValueError('Аргумент data должен иметь тип list')
        if not isinstance(target, list):
            raise ValueError('Аргумент target должен иметь тип list')
        print('Данные инициализированы!')
        self._set_labels_prior_proba(data, target)
        print(f'Априорные вероятности классов {self.class_labels_proba}')
        self._set_word_prior_proba(data, target)
        print('Обучили априорные вероятности слов')
        

    def _predict_proba(self, data: list) -> List[tuple]:
        """Предсказываем класс для текстовой смс

        :param data: массив документов, для каждого из которых нужно предсказать метку
        :return: вероятности для каждого из классов
        """
        prediction = []
        for obj in data:
            posterior_class_proba = defaultdict(lambda: 1)
            for token in self._tokenize_text(obj):
                for label in self.labels:
                    posterior_class_proba[label] *= self.prior_word_proba[label][token]
            # сохраняем для каждой метки класса - сколько меток, таков и размер uple
            prediction.append(
                tuple(
                    posterior_class_proba[label] for label in self.labels
                )
            )
        print(f'proba: {prediction}')
        return prediction
    
    def predict(self, data) -> List[str]:
        predict_labels = []
        for proba in self._predict_proba(data):
            predict_labels.append(self.labels[np.argmax(proba)])
        return predict_labels

naive_bayes = NaiveBayes()

In [42]:
naive_bayes.fit(
    data=df['sms_text'].values.tolist(),
    target=df['class'].tolist()
)

Данные инициализированы!


  0%|          | 13/9661 [00:00<01:14, 128.94it/s]

Априорные вероятности классов {'ham': 0.8658886894075404, 'spam': 0.1341113105924596}


100%|██████████| 9661/9661 [01:11<00:00, 134.37it/s]

Обучили априорные вероятности слов


In [43]:
naive_bayes.prior_word_proba['ham']['thank'], naive_bayes.prior_word_proba['spam']['thank']

(0.019075264358283228, 0.0214190093708166)

Предсказание метки класса

In [44]:
import numpy as np
# рандомный объект датасета

random_obj_ind = np.random.randint(low=0, high=num_objects, size=3)
random_obj_list = df['sms_text'].values[random_obj_ind].tolist()
random_target = df['class'][random_obj_ind].tolist()

print(random_obj_list)
naive_bayes.predict(
    random_obj_list
)

['I cant pick the phone right now. Pls send a message', 'I want to be there so i can kiss you and feel you next to me', "So there's a ring that comes with the guys costumes. It's there so they can gift their future yowifes. Hint hint"]
proba: [(2.7689631504068686e-14, 9.892065334484763e-14), (6.399152575535092e-15, 0.0), (3.71051403289896e-42, 0.0)]


['spam', 'ham', 'ham']